# Module 3: AI Agents with Semantic Kernel
## Building Intelligent AI Agents

### 1. Introduction to SK Agents
Agents in Semantic Kernel are AI-powered entities that can engage in conversations, make decisions, and execute tasks. They can work independently or collaborate in groups to achieve complex goals.

### 2. Creating Basic Agents

Let's start with a simple agent setup:

```python
from semantic_kernel.agents import ChatCompletionAgent
from semantic_kernel.kernel import Kernel
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion
from semantic_kernel.contents.chat_history import ChatHistory

# Create a kernel and add a chat service
kernel = Kernel()
kernel.add_service(AzureChatCompletion(service_id="agent"))

# Create a simple agent with personality
agent = ChatCompletionAgent(
    service_id="agent",
    kernel=kernel,
    name="Pirate",
    instructions="You are a friendly pirate who always speaks in pirate dialect and ends messages with a parrot sound."
)

# Create chat history and helper function for interaction
chat = ChatHistory()

async def chat_with_agent(agent: ChatCompletionAgent, message: str):
    """Function to handle agent interaction"""
    chat.add_user_message(message)
    print(f"User: {message}")
    
    # Use streaming for responsive interaction
    chunks = []
    async for chunk in agent.invoke_stream(chat):
        chunks.append(chunk)
        print(chunk.content, end="", flush=True)  # Show response as it comes
    print("\n")  # New line after response
    
    # Add complete response to chat history
    complete_response = "".join([chunk.content for chunk in chunks])
    chat.add_assistant_message(complete_response)

# Example usage
async def main():
    print("Starting chat with Pirate Agent...")
    
    # Test different types of interactions
    await chat_with_agent(agent, "Hello! Can you help me find treasure?")
    await chat_with_agent(agent, "What's the best way to navigate at sea?")
    await chat_with_agent(agent, "Tell me about your parrot!")

# Run the chat
if __name__ == "__main__":
    asyncio.run(main())
```

### 3. Agents with Plugins
Agents become more powerful when they can use plugins. Here's how to create an agent with custom capabilities:

```python
from typing import Annotated
from semantic_kernel.functions.kernel_function_decorator import kernel_function
from semantic_kernel.connectors.ai.function_choice_behavior import FunctionChoiceBehavior

class WeatherPlugin:
    """Plugin for weather-related functions"""
    
    @kernel_function(description="Get the current weather for a location.")
    def get_weather(
        self,
        location: Annotated[str, "The city name"]
    ) -> str:
        # In real implementation, this would call a weather API
        return f"The weather in {location} is sunny and 22°C"

    @kernel_function(description="Get the weather forecast for next 3 days.")
    def get_forecast(
        self,
        location: Annotated[str, "The city name"]
    ) -> str:
        return f"3-day forecast for {location}: Sunny, Cloudy, Rain"

# Set up kernel with plugins
kernel = Kernel()
kernel.add_service(AzureChatCompletion(service_id="agent"))

# Configure function auto-invocation
settings = kernel.get_prompt_execution_settings_from_service_id(service_id="agent")
settings.function_choice_behavior = FunctionChoiceBehavior.Auto()

# Add plugin to kernel
kernel.add_plugin(WeatherPlugin(), plugin_name="weather")

# Create agent with access to plugins
agent = ChatCompletionAgent(
    service_id="agent",
    kernel=kernel,
    name="WeatherAssistant",
    instructions="""You help users with weather-related queries.
    Always aim to provide the most accurate and detailed information possible.
    When appropriate, combine current weather with forecast information.""",
    execution_settings=settings
)

async def weather_assistant_demo():
    chat = ChatHistory()
    
    async def ask_weather(question: str):
        chat.add_user_message(question)
        print(f"User: {question}")
        
        async for response in agent.invoke_stream(chat):
            print(response.content, end="", flush=True)
        print("\n")
    
    # Test various weather queries
    await ask_weather("What's the weather like in Seattle?")
    await ask_weather("Should I pack an umbrella for my trip to London next week?")
    await ask_weather("Compare the weather in New York and Tokyo.")

# Run the weather assistant
if __name__ == "__main__":
    asyncio.run(weather_assistant_demo())
```

### 4. Group Chat Agents
One of the most powerful features is the ability to create agent groups that can collaborate:

```python
from semantic_kernel.agents import AgentGroupChat
from semantic_kernel.agents.strategies.termination.termination_strategy import TerminationStrategy

# Create specialized agents
researcher = ChatCompletionAgent(
    service_id="researcher",
    kernel=kernel,
    name="Researcher",
    instructions="""
    You are a thorough researcher who:
    - Gathers comprehensive information
    - Focuses on accuracy and details
    - Cites sources when possible
    - Maintains objectivity
    """
)

analyst = ChatCompletionAgent(
    service_id="analyst",
    kernel=kernel,
    name="Analyst",
    instructions="""
    You are an analytical expert who:
    - Reviews information critically
    - Identifies patterns and trends
    - Provides actionable insights
    - Questions assumptions
    """
)

# Create a termination strategy
class ConsensusStrategy(TerminationStrategy):
    """Terminates when agents reach a consensus"""
    
    async def should_agent_terminate(self, agent, history):
        if len(history) < 2:
            return False
        
        last_messages = history[-2:]
        return "agree" in last_messages[-1].content.lower()

# Create the group chat
chat = AgentGroupChat(
    agents=[researcher, analyst],
    termination_strategy=ConsensusStrategy(maximum_iterations=10)
)

async def research_team_demo():
    print("Starting Research Team Analysis...\n")
    
    # Add initial research question
    research_question = """
    Analyze the impact of remote work on company culture and productivity.
    Consider both positive and negative effects.
    """
    
    # Start the group chat
    await chat.add_chat_message(ChatMessageContent(
        role=AuthorRole.USER,
        content=research_question
    ))
    print(f"Research Question: {research_question}\n")
    
    # Let the agents collaborate
    async for message in chat.invoke():
        print(f"{message.name}: {message.content}\n")
    
    print(f"Analysis Complete - Consensus Reached: {chat.is_complete}")

# Run the research team
if __name__ == "__main__":
    asyncio.run(research_team_demo())
```

### 5. Advanced Agent Strategies

#### 5.1 Custom Selection Strategies
Control which agent speaks next:

```python
from semantic_kernel.functions.kernel_function_from_prompt import KernelFunctionFromPrompt
from semantic_kernel.agents.strategies import KernelFunctionSelectionStrategy

selection_function = KernelFunctionFromPrompt(
    function_name="selection",
    prompt="""
    Determine who should speak next based on these rules:
    - After user input, Researcher speaks first
    - After Researcher findings, Analyst reviews
    - After Analyst insights, discuss until consensus
    
    Participants:
    - Researcher
    - Analyst
    
    Chat history:
    {{$history}}
    
    Who should speak next?
    """
)

selection_strategy = KernelFunctionSelectionStrategy(
    function=selection_function,
    kernel=kernel,
    result_parser=lambda result: str(result.value[0]),
    history_variable_name="history"
)
```

#### 5.2 Structured Outputs
Make agents return structured data:

```python
from pydantic import BaseModel
from semantic_kernel.kernel_pydantic import KernelBaseModel

class AnalysisResult(KernelBaseModel):
    """Structured output for analysis"""
    main_points: list[str]
    confidence_score: float
    recommendations: list[str]

class StructuredAnalysisAgent:
    """Agent that provides structured analysis results"""
    
    def __init__(self, kernel: Kernel):
        self.agent = ChatCompletionAgent(
            service_id="analyst",
            kernel=kernel,
            name="StructuredAnalyst",
            instructions="""
            Analyze information and respond in JSON format matching this schema:
            {
                "main_points": ["point1", "point2", ...],
                "confidence_score": float between 0-1,
                "recommendations": ["rec1", "rec2", ...]
            }
            
            Always ensure:
            - Main points are clear and concise
            - Confidence score reflects analysis certainty
            - Recommendations are actionable
            """
        )
        
    async def analyze(self, topic: str) -> AnalysisResult:
        """Perform analysis and return structured results"""
        chat = ChatHistory()
        chat.add_user_message(topic)
        
        # Get response from agent
        async for response in self.agent.invoke(chat):
            try:
                # Parse JSON response into our model
                return AnalysisResult.model_validate_json(response.content)
            except Exception as e:
                print(f"Error parsing response: {e}")
                return None

async def structured_analysis_demo():
    # Set up the agent
    kernel = Kernel()
    kernel.add_service(AzureChatCompletion(service_id="analyst"))
    analyst = StructuredAnalysisAgent(kernel)
    
    # Test topics for analysis
    topics = [
        "The impact of AI on job markets in the next 5 years",
        "The future of renewable energy adoption",
        "Trends in remote work and digital collaboration"
    ]
    
    for topic in topics:
        print(f"\nAnalyzing: {topic}")
        result = await analyst.analyze(topic)
        
        if result:
            print("\nAnalysis Results:")
            print(f"Confidence Score: {result.confidence_score}")
            print("\nMain Points:")
            for point in result.main_points:
                print(f"- {point}")
            print("\nRecommendations:")
            for rec in result.recommendations:
                print(f"- {rec}")
        else:
            print("Analysis failed")

# Run the structured analysis demo
if __name__ == "__main__":
    asyncio.run(structured_analysis_demo())
```

### 6. Best Practices

1. **Agent Design**
   - Give clear, specific instructions
   - Define scope and limitations
   - Include example interactions
   - Specify output formats

2. **Group Dynamics**
   - Assign distinct roles
   - Define clear interaction patterns
   - Implement proper termination conditions
   - Handle disagreements

3. **Error Handling**
```python
async def safe_agent_invoke(agent: ChatCompletionAgent, chat: ChatHistory):
    try:
        async for response in agent.invoke_stream(chat):
            yield response
    except Exception as e:
        yield f"Error: {str(e)}"
        chat.add_system_message("Error occurred during processing")
```

4. **Performance**
   - Limit maximum iterations
   - Use streaming for long responses
   - Implement timeouts
   - Cache frequently used results

### 7. Advanced Use Cases

1. **Review System**
```python
class ReviewSystem:
    def __init__(self):
        self.author = ChatCompletionAgent(
            name="Author",
            instructions="Create content following style guidelines."
        )
        
        self.reviewer = ChatCompletionAgent(
            name="Reviewer",
            instructions="Review content for quality and guidelines."
        )
        
        self.chat = AgentGroupChat(
            agents=[self.author, self.reviewer],
            termination_strategy=KernelFunctionTerminationStrategy(
                function=self._create_approval_check()
            )
        )
    
    def _create_approval_check(self):
        return KernelFunctionFromPrompt(
            function_name="check_approval",
            prompt="Check if content is approved. Return 'yes' or 'no'."
        )
```

2. **Research Team**
```python
class ResearchTeam:
    def __init__(self):
        self.researcher = ChatCompletionAgent(
            name="Researcher",
            instructions="Gather and verify information."
        )
        
        self.analyst = ChatCompletionAgent(
            name="Analyst",
            instructions="Analyze findings and identify patterns."
        )
        
        self.writer = ChatCompletionAgent(
            name="Writer",
            instructions="Create clear, engaging reports."
        )
        
        self.chat = AgentGroupChat(
            agents=[self.researcher, self.analyst, self.writer],
            selection_strategy=self._create_selection_strategy()
        )
```

### Resources
- [Semantic Kernel Agents Documentation](https://learn.microsoft.com/semantic-kernel/agents)
- [GitHub Examples](https://github.com/microsoft/semantic-kernel/tree/main/python/samples/agents)
- [Best Practices Guide](https://learn.microsoft.com/semantic-kernel/agents/best-practices)